# Lab 1.2 Pipeline Infrastrucure 

### List of Steps

> Import Required Modules

> Setting up PostGIS connection and creating geometric primitives (polygon)

> Installing PostGIS on Virtual Machine with configured Firewall and CPUs and with Github web_server repo via Dr. Runck's repo

> Using an app.py to create and establish connection from psycopg2/PostGIS server to HTTP port to display GeoJSON and then save to local drive

> From local drive, use Google Cloud Run to continuously update resources/information on ArcGIS Online/servers for potential clients

> (Alternative) Convert GeoJSON information to JSON and JSON strins in dictionary formats and display on ArcGIS Online as static object

In [4]:
import arcpy
import psycopg2
from psycopg2 import sql
from psycopg2.extras import RealDictCursor

In [9]:
# TEST
#connection= psycopg2.connect(host = '107.178.210.122',
                             #database = 'lab0',
                             #user = 'postgres',
                             #password = '----')
#cursor = connection.cursor()

#cursor = connection.cursor()
#create_table_query = sql.SQL("CREATE TABLE Table2 (polygon geometry)")
#cursor.execute(create_table_query)

#connection.commit()
#cursor.close()
#connection.close()

In [26]:
# TEST
#points = arcpy.Array([arcpy.Point(1,16),
                          #arcpy.Point(51,2),
                         # arcpy.Point(23,32),
                          #arcpy.Point(7,4)])

#polygon = arcpy.Polygon(points)
#polygon_wkt = polygon.WKT
# - tmux - terminal multiplex for Lab 2 let flask run in background

In [29]:
# Create a polygon using ArcPy geometry primitives
polygon = arcpy.Polygon(arcpy.Array([arcpy.Point(1,16),
                          arcpy.Point(51,2),
                          arcpy.Point(23,32),
                          arcpy.Point(7,4)]))

# Convert the polygon to WKT
wkt = polygon.WKT

# Connect to the PostGIS database using psycopg2
connection = psycopg2.connect(host = '107.178.210.122',
                             database = 'lab0',
                             user = 'postgres',
                             password = 'ARDGISumn1994123')

# Create a cursor
cursor = connection.cursor()

# Define the insert statement
insert_stmt = "INSERT INTO table2 (polygon) VALUES (ST_GeomFromText('{}', 4326))".format(wkt)


# Execute the insert statement
cursor.execute(insert_stmt)

# Commit the transaction
connection.commit()

# Close the cursor and the database connection
cursor.close()
connection.close()


In [5]:
#Install PostGIS and psycopg2 on the VM:
sudo apt-get update
sudo apt-get install postgis
pip install psycopg2-binary
pip install Flask

SyntaxError: invalid syntax (<string>, line 2)

In [ ]:
#make a .py file of the following file and upload to VM ssh

import psycopg2
from flask import Flask, jsonify
from psycopg2 import sql
from psycopg2.extras import RealDictCursor

app = Flask(__name__)

@app.route('/get_polygon', methods=['GET'])
def get_polygon():
    conn= psycopg2.connect(host = '35.223.186.20',
                             database = 'lab0',
                             user = 'postgres',
                             password = '')

    cur = connection.cursor(cursor_factory=RealDictCursor)
    cur.execute(sql.SQL("SELECT mytable, ST_AsGeoJSON(geom)::json AS geometry FROM mytable WHERE id = %s"), (1,))
    result = cur.fetchone()
    cur.close()
    conn.close()
    return jsonify(result)

#name the .py file "app.py", then upload this app.py file to your ssh on google VM

In [ ]:
#Replace your_vm_external_ip with the external IP address of your VM.
export FLASK_APP=app6.py
flask run --host=0.0.0.0

In [30]:
import requests

url = "http://34.69.52.173:5000/get_polygon"
response = requests.get(url)

if response.status_code == 200:
    geojson = response.json()
    with open("C:/Users/Alexander Danielson/Desktop/New folder/my_geojson_file.geojson", "w") as f:
        f.write(str(geojson))
        print("GeoJSON file saved to C:/my_geojson_file.geojson")
else:
    print("Failed to retrieve geojson data from the URL.")



GeoJSON file saved to C:/my_geojson_file.geojson


In [ ]:
# TEST
#import arcpy

# Set up connection to ArcGIS Online
#gis = arcpy.gis.GIS("https://www.arcgis.com", "", "password")

# Create a new item for the feature layer
#item_properties = {'title': 'My Feature Layer', 'type': 'GeoJson'}
#item = gis.content.add(item_properties)

# Publish the feature layer from the GeoJSON file
#feature_layer = item.publish(file_path=r'C:\path\to\my\file.geojson')

# Share the feature layer
#feature_layer.share(everyone=True)

# Print the URL to the feature layer
#print(feature_layer.url)

In [31]:
import json

# First JSON
json_string_1 = '{"geometry":{"coordinates":[[[[1.00012207,1.00012207],[2.00012207,1.00012207],[2.00012207,2.00012207],[1.00012207,2.00012207],[1.00012207,1.00012207]]]],"type":"MultiPolygon"},"table2":"(0106000020E61000000100000001030000000100000005000000000000008000F03F000000008000F03F0000000040000040000000008000F03F00000000400000400000000040000040000000008000F03F0000000040000040000000008000F03F000000008000F03F)"}'

# Load the JSON string as a Python object
json_obj_1 = json.loads(json_string_1)

# Convert the first JSON object to the second JSON object
json_obj_2 = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": json_obj_1['geometry'],
            "properties": {
                "table2": json_obj_1['table2']
            }
        }
    ]
}

# Convert the Python object to a JSON string
json_string_2 = json.dumps(json_obj_2)

# Print the resulting JSON string
print(json_string_2)

{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"coordinates": [[[[1.00012207, 1.00012207], [2.00012207, 1.00012207], [2.00012207, 2.00012207], [1.00012207, 2.00012207], [1.00012207, 1.00012207]]]], "type": "MultiPolygon"}, "properties": {"table2": "(0106000020E61000000100000001030000000100000005000000000000008000F03F000000008000F03F0000000040000040000000008000F03F00000000400000400000000040000040000000008000F03F0000000040000040000000008000F03F000000008000F03F)"}}]}


In [34]:
# Save the dictionary as a JSON file
with open('C:/Users/Alexander Danielson/Desktop/New folder/esri_json1.json', 'w') as json_file:
    json.dump(json_obj_2, json_file)

In [35]:
import arcpy
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Connect to your ArcGIS Online organization
gis = GIS("https://www.arcgis.com", "ARDPHOTO", "----")

# Upload the GeoJSON file as a new item
item = gis.content.add({'type': 'GeoJson'}, 'C:/Users/Alexander Danielson/Desktop/New folder/esri_json1.json')

# Publish the new item as a feature layer
feature_layer_item = item.publish()

# Get the feature layer URL and print it
feature_layer_url = feature_layer_item.url
print(feature_layer_url)

Exception: Unable to publish item.
User 'ARDPHOTO' does not have publishing privileges.
(Error Code: 400)